[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ryansteed/ieat/blob/master/notebooks/experiments.ipynb)

# Image GPT Bias
**Image Embedding Association Test**

Ryan Steed

This script adapted from https://colab.research.google.com/github/apeguero1/image-gpt/blob/master/Transformers_Image_GPT.ipynb.


In [126]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

%cd /Users/steed/caliskan/git/ieat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/steed/caliskan/git/ieat


## Download Model

In [6]:
!nvidia-smi #OpenAI says you need 16GB GPU for the large model, but it may work if you lower n_sub_batch on the others.

zsh:1: command not found: nvidia-smi


In [10]:
import urllib

# don't share these to Github or elsewhere!
u = "your_username"
p = urllib.parse.quote("your_password")
!git clone --recurse-submodules -j8 https://$u:$p@github.com/$u/ieat.git
%cd ieat

Cloning into 'ieat'...
remote: Invalid username or password.
fatal: Authentication failed for 'https://github.com/your_username/ieat.git/'
/Users/steed/caliskan/git/ieat


In [7]:
# note - you may need to restart the kernel for these installations to take effect
!pip install -e .
!pip install -e weat

Obtaining file:///content/ieat
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 890kB 17.7MB/s 
     |████████████████████████████████| 1.1MB 16.3MB/s 
     |████████████████████████████████| 3.0MB 29.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3f961765e90189d01825b152c097d0f8e6956b7800c4cc9e041147499cc27419
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Running setup.py develop for ieat
Obtaining file:///content/ieat/weat
  Running setup.py develop for weat


In [9]:
model_sizes = ["s", "m", "l"] #small medium large, xl not available
model_size = "l"
models_dir = "models"
color_clusters_dir = "clusters"
n_px = 32
depth = 152
width = 3
sk = 1

In [ ]:
# download the model - skip if already downloaded
!python image-gpt/download.py --model {model_size} --ckpt 1000000 --clusters --download_dir {models_dir}/{model_size}
!python image-gpt/download.py --clusters --download_dir {color_clusters_dir}

input args:
 {
    "download_dir":"models/l",
    "model":"l",
    "ckpt":"1000000",
    "clusters":true,
    "dataset":null
}
Fetching model.ckpt-1000000.data-00000-of-00064: 1.00kit [00:00, 777kit/s]      
Fetching model.ckpt-1000000.data-00001-of-00064: 252Mit [00:06, 37.4Mit/s]      
Fetching model.ckpt-1000000.data-00002-of-00064:  87%|▊| 217M/249M [00:06<00:00,

## IATs

### Quick Summary

In [127]:
from ieat.api import test_all

import pandas as pd
import numpy as np

results = test_all(
    {
        'igpt-logit': (model_size,models_dir,color_clusters_dir,n_px),
        'igpt': (model_size,models_dir,color_clusters_dir,n_px),
        'simclr': (depth, width, sk)
    }
)

# igpt-logit #
## Insect-Flower ##


2020-10-05 21:48:41,393 [19618] WARNING  root:225: [JupyterRequire] Equalities contributed 1/924 to p-value


## Weapon ##
## Weapon (Modern) ##


2020-10-05 21:48:41,493 [19618] WARNING  root:225: [JupyterRequire] Equalities contributed 1/924 to p-value
2020-10-05 21:48:41,650 [19618] WARNING  root:225: [JupyterRequire] Equalities contributed 1/10000 to p-value


## Native ##
## Asian ##


...

## Sexuality ##
## Race ##


2020-10-05 21:49:14,938 [19618] WARNING  root:225: [JupyterRequire] Equalities contributed 1/924 to p-value


## Arab-Muslim ##
## Age ##


2020-10-05 21:49:17,207 [19618] WARNING  root:225: [JupyterRequire] Equalities contributed 1/924 to p-value


## Gender-Science ##
## Gender-Career ##
## Intersectional-Gender-Science-MF ##
## Intersectional-Gender-Science-WMBM ##
## Intersectional-Gender-Science-WMBF ##
## Intersectional-Gender-Science-WMWF ##
## Intersectional-Gender-Career-MF ##
## Intersectional-Gender-Career-WMBM ##
## Intersectional-Gender-Career-WMBF ##
## Intersectional-Gender-Career-WMWF ##
## Intersectional-Valence-BW ##
## Intersectional-Valence-WMBM ##
## Intersectional-Valence-WMBF ##
## Intersectional-Valence-WMWF ##
## Intersectional-Valence-WFBM ##
## Intersectional-Valence-BFBM ##
## Intersectional-Valence-WFBF ##
## Intersectional-Valence-FM ##


In [128]:
import pandas as pd

results_df = pd.DataFrame(results).transpose()
results_df.columns = ["X", "Y", "A", "B", "d", "p", "n_t", "n_a"]
for c in results_df.columns[:4]:
    results_df[c] = results_df[c].str.split("/").str[-1]
results_df["sig"] = ""
for l in [0.10, 0.05, 0.01]:
    results_df.sig[results_df.p < l] += "*"
intersectional = results_df.index.get_level_values(0).str.contains("Intersectional")
gender = results_df.index.get_level_values(0).str.contains("Gender")
results_df[gender]

,,X,Y,A,B,d,p,n_t,n_a,sig
Gender-Science,igpt-logit,male,female,science,liberal-arts,0.36811,0.0512,40,21,*
Gender-Career,igpt-logit,male,female,career,family,0.0369452,0.4262,40,21,
Intersectional-Gender-Science-MF,igpt-logit,male,female,science,liberal-arts,0.622647,0.0026,40,21,***
Intersectional-Gender-Science-WMBM,igpt-logit,white-male,black-male,science,liberal-arts,0.282217,0.19,20,21,
Intersectional-Gender-Science-WMBF,igpt-logit,white-male,black-female,science,liberal-arts,0.809627,0.0044,20,21,***
Intersectional-Gender-Science-WMWF,igpt-logit,white-male,white-female,science,liberal-arts,0.835792,0.003,20,21,***
Intersectional-Gender-Career-MF,igpt-logit,male,female,career,family,-0.293549,0.9072,40,21,
Intersectional-Gender-Career-WMBM,igpt-logit,black-male,white-male,career,family,0.217328,0.2561,20,21,
Intersectional-Gender-Career-WMBF,igpt-logit,white-male,black-female,career,family,-0.598387,0.9722,20,21,
Intersectional-Gender-Career-WMWF,igpt-logit,white-male,white-female,career,family,-0.332725,0.844,20,21,


In [129]:
# Generate latex table output
experiments = (
    "Insect-Flower",
    "Gender-Science",
    "Gender-Career",
    "Skin-Tone",
    "Race",
    "Weapon",
    "Weapon (Modern)",
    "Native",
    "Asian",
    "Weight",
    "Religion",
    "Sexuality",
    "Disability",
    "Arab-Muslim",
    "Age"
)
latex_df = results_df
latex_df.index = latex_df.index.set_names(["Test", "Model"])
latex_df = latex_df.sort_index(level=["Test", "Model"])
experiments += tuple(results_df.index.get_level_values("Test")[results_df.index.get_level_values("Test").str.contains("Intersectional")])
def round(s, i='{0:.2f}', sci=False):
    return s.apply(
        lambda p: i.format(p) if not sci else (
            "$<10^{-3}$" if p<=0.001 else
            "$<10^{-2}$" if p<=0.01 else
            i.format(p)
        )
    ).astype(str)
latex_df["d"] = round(latex_df.d) #+ latex_df.sig.astype(str)
latex_df["p"] = round(latex_df.p, sci=True)
latex_df = latex_df.drop(columns=["sig"])

original_results = {
    "Age": 1.23,
    "Arab-Muslim": 0.33,
    "Asian": 0.62,
    "Disability": 1.05,
    "Gender-Career": 1.10,
    "Gender-Science": 0.93,
    "Insect-Flower": 1.35,
    "Native": 0.46,
    "Race": 0.86,
    "Religion": -0.34,
    "Sexuality": 0.74,
    "Skin-Tone": 0.73,
    "Weapon": 1.00,
    "Weight": 1.83
}
original = pd.DataFrame.from_dict(original_results, orient='index', columns=["IAT $d$"])
latex_df.columns = [f"${col}$" for col in latex_df.columns]
original.index = original.index.set_names(["Test"])
latex_df = latex_df.join(original)

def models(df, *keys):
    return df["Model"].isin(keys)

# some filters
latex_df = latex_df.reset_index(level="Model")
main_experiment = latex_df.index.get_level_values("Test").isin(experiments)
intersectional = latex_df.index.get_level_values("Test").str.contains("Intersectional")

# some formatters
def color_d(x):
    d = float(x)
    x = str(x)
    return \
        f"\cellcolor{{d_large}}{x}" if d > 0.8 else\
        f"\cellcolor{{d_medium}}{x}" if d > 0.5 else\
        f"\cellcolor{{d_small}}{x}" if d > 0.2 else\
        x

def model_fmt(x):
    return "iGPT" if x == "igpt" else "iGPT Logit" if x == "igpt-logit" else "SimCLR"

class FMT:
    def __init__(self):
        self.last = None
        self.i = 0
        self.noskips = True
    def test_fmt(self, x):
        if self.noskips and x.startswith("Intersectional"):
            b = x.strip("Intersectional-").split("-")
            i = "-".join(b[:-1])
            return f"{i} ({b[-1]})"
        if x == self.last:
            return ""
        self.last = x
        if x in ["Age", "Disability", "Race", "Skin-Tone", "Weight"]:
            x += "\textsuperscript{\textdagger}"
        if x in ["Asian", "Native", "Weapon"]:
            x += "\textsuperscript{$\mathsection$}"
        return x
    def category_format(self, x):
        if self.noskips and x == self.last and self.i < 1 and x not in ["white-male", "black-male", "white-female", "black-female"]:
            self.i += 1
            return ""
        else:
            self.last = x
            self.i = 0
        x = x.capitalize()
        cats_fmt = {
            "Other-people": "Other",
            "arab-muslim": "Arab-Muslim",
            "European-american": "European American",
            "Asian-american": "Asian American",
            "Euro": "European American",
            "Native": "Native American",
            "African-american": "African American",
            "Liberal-arts": "Liberal Arts",
            "Us": "U.S.",
            "Tool-modern": "Tool (Modern)",
            "Weapon-modern": "Weapon (Modern)",
            "Black-female": "Black Female",
            "Black-male": "Black Male",
            "White-female": "White Female",
            "White-male": "White Male"
        }
        return cats_fmt.get(x) if cats_fmt.get(x) is not None else x
    def n_fmt(self, x):
        if self.noskips and x == self.last and self.i < 1:
            self.i += 1
            return ""
        else:
            self.last = x
            self.i = 0
            return x

fmter = FMT()
    
# main results
cols = ["$X$", "$Y$", "$A$", "$B$", "$n_t$", "$n_a$", "Model", "$d$", "$p$", "IAT $d$"]
results_table = latex_df[main_experiment & ~intersectional & models(latex_df, "igpt", "simclr")][cols]
display(results_table)
results_table.index = [fmter.test_fmt(x) for x in results_table.index]
results_table.to_latex('output/results.tex', escape=False, na_rep="N/A",
    formatters={
        "$d$": color_d, 
        "IAT $d$": color_d,
        "$X$": fmter.category_format,
        "$Y$": fmter.category_format,
        "$A$": fmter.category_format,
        "$B$": fmter.category_format,
        "Model": model_fmt,
        "$n_t$": fmter.n_fmt,
        "$n_a$": fmter.n_fmt
    }
)

# logit results
cols = [c for c in cols if c not in ("Model", "IAT $d$")]
logit_table = latex_df[main_experiment & ~intersectional & models(latex_df, "igpt-logit")][cols]
display(logit_table)
logit_table.index = [fmter.test_fmt(x) for x in logit_table.index]
fmter.noskips = False
logit_table.to_latex('output/logit.tex', escape=False,
    formatters={
        "$d$": color_d, 
        "$X$": fmter.category_format,
        "$Y$": fmter.category_format,
        "$A$": fmter.category_format,
        "$B$": fmter.category_format,
        "$n_t$": fmter.n_fmt,
        "$n_a$": fmter.n_fmt
    }
)

# intersectional results
cols = ["$X$", "$Y$", "$A$", "$B$", "$n_t$", "$n_a$", "$d$", "$p$"]
fmter.noskips = True
intersectional_table = latex_df[main_experiment & intersectional & models(latex_df, "igpt")][cols]
display(intersectional_table)
intersectional_table.index = [fmter.test_fmt(x) for x in intersectional_table.index]
intersectional_table.to_latex('output/intersectional.tex', escape=False, 
    formatters={
        "$d$": color_d, 
        "$X$": fmter.category_format,
        "$Y$": fmter.category_format,
        "$A$": fmter.category_format,
        "$B$": fmter.category_format,
        "Test": intersectional_fmt
#         "$n_t$": fmter.n_fmt,
#         "$n_a$": fmter.n_fmt
    }
)

# table of iats w/ categories
# iats_table = latex_df.groupby("Test").head(1)\
#     .drop(columns=["$d$", "$p$", "$n_t$", "$n_a$", "Model"])
# display(iats_table)
# iats_table.to_latex('output/iats.tex', escape=False, multirow=True)

# stimuli table
stimuli = pd.read_csv("data/stimuli.csv")
stimuli.dropna(axis=0, subset=["Word", "Source"])\
    .drop(columns="Collected (N=n)").set_index(["IAT", "Category"]).to_latex("output/stimuli.tex")

,$X$,$Y$,$A$,$B$,$n_t$,$n_a$,Model,$d$,$p$,IAT $d$
Test,,,,,,,,,,
Age,young,old,pleasant,unpleasant,6,55,igpt,0.42,0.24,1.23
Age,young,old,pleasant,unpleasant,6,55,simclr,0.59,0.16,1.23
Arab-Muslim,other-people,arab-muslim,pleasant,unpleasant,10,55,igpt,0.86,0.03,0.33
Arab-Muslim,other-people,arab-muslim,pleasant,unpleasant,10,55,simclr,1.06,$<10^{-2}$,0.33
Asian,european-american,asian-american,american,foreign,6,6,igpt,0.25,0.34,0.62
Asian,european-american,asian-american,american,foreign,6,6,simclr,0.47,0.21,0.62
Disability,disabled,abled,pleasant,unpleasant,4,55,igpt,-0.02,0.53,1.05
Disability,disabled,abled,pleasant,unpleasant,4,55,simclr,0.38,0.34,1.05
Gender-Career,male,female,career,family,40,21,igpt,0.62,$<10^{-2}$,1.10


,$X$,$Y$,$A$,$B$,$n_t$,$n_a$,$d$,$p$
Test,,,,,,,,
Age,young,old,pleasant,unpleasant,6,55,0.38,0.38
Arab-Muslim,other-people,arab-muslim,pleasant,unpleasant,10,55,0.06,0.43
Asian,european-american,asian-american,american,foreign,6,6,0.25,0.36
Disability,disabled,abled,pleasant,unpleasant,4,55,-0.65,0.76
Gender-Career,male,female,career,family,40,21,0.04,0.43
Gender-Science,male,female,science,liberal-arts,40,21,0.37,0.05
Insect-Flower,flower,insect,pleasant,unpleasant,35,55,-0.32,0.92
Native,euro,native,us,world,8,5,0.32,0.27
Race,european-american,african-american,pleasant,unpleasant,6,55,-0.17,0.62


,$X$,$Y$,$A$,$B$,$n_t$,$n_a$,$d$,$p$
Test,,,,,,,,
Intersectional-Gender-Career-MF,male,female,career,family,40,21,0.81,$<10^{-3}$
Intersectional-Gender-Career-WMBF,white-male,black-female,career,family,20,21,0.20,0.27
Intersectional-Gender-Career-WMBM,black-male,white-male,career,family,20,21,0.89,$<10^{-2}$
Intersectional-Gender-Career-WMWF,white-male,white-female,career,family,20,21,0.97,$<10^{-3}$
Intersectional-Gender-Science-MF,male,female,science,liberal-arts,40,21,-0.00,0.50
Intersectional-Gender-Science-WMBF,white-male,black-female,science,liberal-arts,20,21,0.80,$<10^{-2}$
Intersectional-Gender-Science-WMBM,white-male,black-male,science,liberal-arts,20,21,0.49,0.06
Intersectional-Gender-Science-WMWF,white-male,white-female,science,liberal-arts,20,21,-0.37,0.88
Intersectional-Valence-BFBM,black-female,black-male,pleasant,unpleasant,20,55,0.17,0.29


### Randomized Insect-Flower

In [58]:
from ieat.api import test

print(test(
    'data/experiments/insect-flower/flower', 'data/experiments/insect-flower/insect', 
    'data/experiments/valence/pleasant', 'data/experiments/valence/unpleasant',
    model_type='igpt',
    model_params=(model_size,models_dir,color_clusters_dir,n_px),
    randomized=False
))
ps = []
for i in range(1000):
    if i % 10 == 0: print(i if i % 50 == 0 else ".", end=" ")
    _, p = test(
        'data/experiments/insect-flower/flower', 'data/experiments/insect-flower/insect', 
        'data/experiments/valence/pleasant', 'data/experiments/valence/unpleasant',
        model_type='igpt',
        model_params=(model_size,models_dir,color_clusters_dir,n_px),
        randomized=True
    )
    ps.append(p)
print("Proportion of p-values under 0.1: ", len([p for p in ps if p < 0.1])/1000.)
print("Proportion of p-values under 0.01: ", len([p for p in ps if p < 0.01])/1000.)

(0.3440325875113741, 0.0736)
0 . . . . 50 . . . . 100 . . . . 150 . . . . 200 . . . . 250 . . . . 300 . . . . 350 . . . . 400 . . . . 450 . . . . 500 . . . . 550 . . . . 600 . . . . 650 . . . . 700 . . . . 750 . . . . 800 . . . . 850 . . . . 900 . . . . 950 . . . . Proportion of p-values under 0.1:  0.112
Proportion of p-values under 0.01:  0.015


### Detail View

#### Weapons

In [ ]:
from ieat.api import test

test(
    "data/experiments/weapon/black",
    "data/experiments/weapon/white",
    "data/experiments/weapon/tool",
    "data/experiments/weapon/weapon",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Native

In [ ]:
test(
    "data/experiments/native/euro",
    "data/experiments/native/native",
    "data/experiments/native/us",
    "data/experiments/native/world",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Asian

In [ ]:
test(
    "data/experiments/asian/european-american",
    "data/experiments/asian/asian-american",
    "data/experiments/asian/american",
    "data/experiments/asian/foreign",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px
#     from_cache=False
)

#### Insect-Flower

In [ ]:
from ieat.api import test

test(
    "data/experiments/insect-flower/flower",
    "data/experiments/insect-flower/insect",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Weight

In [ ]:
from ieat.api import test

test(
    "data/experiments/weight/thin",
    "data/experiments/weight/fat",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Skin-Tone

In [ ]:
from ieat.api import test

test(
    "data/experiments/skin-tone/light",
    "data/experiments/skin-tone/dark",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Disability

In [ ]:
from ieat.api import test

test(
    "data/experiments/disabled/disabled",
    "data/experiments/disabled/abled",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Presidents

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/kennedy",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/clinton",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/bush",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

In [ ]:
from ieat.api import test

test(
    "data/experiments/presidents/trump",
    "data/experiments/presidents/lincoln",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Religion

In [ ]:
from ieat.api import test

test(
    "data/experiments/religion/christianity",
    "data/experiments/religion/judaism",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Science

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/science",
    "data/experiments/gender/liberal-arts",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Gender-Career

In [ ]:
from ieat.api import test

test(
    "data/experiments/gender/career",
    "data/experiments/gender/family",
    "data/experiments/gender/male",
    "data/experiments/gender/female",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Sexuality

In [ ]:
from ieat.api import test

test(
    "data/experiments/sexuality/gay",
    "data/experiments/sexuality/straight",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Race

In [ ]:
from ieat.api import test

test(
    "data/experiments/race/african-american",
    "data/experiments/race/european-american",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Arab-Muslim

In [ ]:
from ieat.api import test

test(
    "data/experiments/arab-muslim/other-people",
    "data/experiments/arab-muslim/arab-muslim",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

#### Age

In [ ]:
from ieat.api import test

test(
    "data/experiments/age/young",
    "data/experiments/age/old",
    "data/experiments/valence/pleasant",
    "data/experiments/valence/unpleasant",
    model_type="openai", 
    model_size=model_size,
    models_dir=models_dir,
    clusters_dir=color_clusters_dir,
    n_px=n_px,
#     from_cache=False,
    verbose=True
)

### Download Cached Embeddings

In [ ]:
# to download from colab
from google.colab import files

!zip -r embeddings_colab.zip embeddings
files.download("embeddings_colab.zip")